<a href="https://colab.research.google.com/github/abstractguy/TSO_project/blob/master/TSO_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install cmake xvfb python-opengl ffmpeg libgle3 x11-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [760 kB]
Get

In [2]:
!pip install --upgrade setuptools
!pip install ez_setup gym gym[all] pyvirtualdisplay pyglet==1.3.2 keras-rl

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.2.0)
     |████████████████████████████████| 1.0MB 5.5MB/s 
     |████████████████████████████████| 40kB 6.8MB/s 
  Created wheel for ez-setup: filename=ez_setup-0.9-cp36-none-any.whl size=11014 sha256=dab0aca08e7a26f2837deea896b5376c74801f44c9f150915d8a6c7d79780af8
  Stored in directory: /root/.cache/pip/wheels/dc/e8/6b/3d5ff5a3efd7b5338d1e173ac981771e2628ceb2f7866d49ad
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48379 sha256=0d3daaff0b0f43bda836a295d259af389e02f7116aed59c9d2ca60699c099ad7
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built ez-setup keras-rl
  Found existing installation: pyglet 1.4.2
    Uninstalling pyglet-1.4.2:
      Successfully uninstalled pyglet-1.4.2


In [0]:
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

In [0]:
from glob import glob
from io import open as io_open
from base64 import b64encode
from numpy.random import seed

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
from gym import logger as gymlogger, make as make_env
from gym.wrappers import Monitor

In [0]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [0]:
# env = wrap_env(env)
def show_video():
    mp4list = glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io_open(mp4, 'r+b').read()
        encoded = b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                                    loop controls style="height: 400px;">
                                    <source src="data:video/mp4;base64,{0}" 
                                    type="video/mp4" /> </video>'''.format(
                                        encoded.decode('ascii'))))
    else: 
        print('Could not find video')

In [0]:
def wrap_env(env):
  return Monitor(env, './video', force=True)

In [0]:
ENV_NAME = 'CartPole-v1'
env = wrap_env(make_env(ENV_NAME))
env.seed(123)
seed(123)
nb_actions = env.action_space.n
gymlogger.set_level(40) # Error only.
display = Display(visible=0, size=(1400, 900))
display.start()
%matplotlib inline

In [11]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))
model.summary()

W0831 23:46:42.291037 140304730335104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0831 23:46:42.356542 140304730335104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 23:46:42.398266 140304730335104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                80        
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 34        
Total params: 658
Trainable params: 658
Non-trainable params: 0
_________________________________________________________________


In [0]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()

In [0]:
# Enable the dueling network (dueling_type is one of {'avg', 'max', 'naive'}).
dqn = DQNAgent(model=model, 
               nb_actions=nb_actions, 
               memory=memory, 
               nb_steps_warmup=30,
               enable_dueling_network=True, 
               dueling_type='avg', 
               target_model_update=0.01, 
               policy=policy)

In [14]:
dqn.compile(Adam(lr=0.001), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)
dqn.save_weights('duel_dqn_' + ENV_NAME + '_weights.h5f', overwrite=True)
dqn.test(env, nb_episodes=10, visualize=True)

W0831 23:46:42.656624 140304730335104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0831 23:46:42.657849 140304730335104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0831 23:46:43.009269 140304730335104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Training for 50000 steps ...
    21/50000: episode: 1, duration: 2.742s, episode steps: 21, steps per second: 8, episode reward: 21.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.381 [0.000, 1.000], mean observation: 0.060 [-1.001, 1.740], loss: --, mean_absolute_error: --, mean_q: --


/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


    40/50000: episode: 2, duration: 1.807s, episode steps: 19, steps per second: 11, episode reward: 19.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.737 [0.000, 1.000], mean observation: -0.030 [-2.725, 1.808], loss: 0.466300, mean_absolute_error: 0.525609, mean_q: -0.079362
    80/50000: episode: 3, duration: 0.268s, episode steps: 40, steps per second: 149, episode reward: 40.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.475 [0.000, 1.000], mean observation: -0.027 [-1.102, 0.646], loss: 0.346425, mean_absolute_error: 0.467895, mean_q: 0.113354
   102/50000: episode: 4, duration: 0.093s, episode steps: 22, steps per second: 237, episode reward: 22.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.591 [0.000, 1.000], mean observation: -0.080 [-1.588, 0.776], loss: 0.194776, mean_absolute_error: 0.500565, mean_q: 0.447426
   138/50000: episode: 5, duration: 0.152s, episode steps: 36, steps per second: 237, episode reward: 36.000, mean reward: 1.000 [1.000, 1.000

Error: ignored

In [15]:
# Reexecute this cell after crash.
dqn.done = True
show_video()